
### 01 - Model Predictor
![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


In [ ]:
# Import Libraries
# ===================================================================

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import image
from PIL import Image

In [ ]:
# Define Model and Dataset Paths
# ===================================================================

mypath = '03 - Images'
modelpath = '05 - Malaria_Model.keras'

In [ ]:
# Load Malaria data
# ===================================================================

BATCH_SIZE = 32
IMAGE_SIZE = [200, 200]

ds, info = tfds.load('malaria', split='train[85%:]',
                     shuffle_files=True, with_info=True)


In [ ]:
# Save the Loaded data as images onto our disk
# ===================================================================

# Define a function to save the images
def save_images(dataset, folder_path, limit):
    for i, example in enumerate(dataset):
        # Stop saving images after reaching the limit
        if i >= limit:
            break

        # Get the image from the example
        image = example["image"].numpy()
        
        # Convert the image array to an Image object
        image = Image.fromarray(image)
        
        # Save the image
        image.save(os.path.join(folder_path, f'image_{i}.png'))

# Create a directory if it doesn't exist
if not os.path.exists(mypath):
    os.makedirs(mypath)

# Call the function to save the images
save_images(ds, mypath, 50)

In [ ]:
# Load the Trained Model
# ===================================================================

# Specify the path to your saved model
model_path = modelpath
# Load the model
model2 = tf.keras.models.load_model(model_path)

In [ ]:
# Predict on these saved images
# ===================================================================

folder_path = mypath

# load all images into a list
images = []
image_list = []
for img in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img)
    img = image.load_img(img_path)
    
    # ----------------------------------------------------
    # Add an extra dimension (for batch size)
    img = tf.expand_dims(img, axis=0)
    # Define the target size
    target_size = [200, 200]
    # Crop or pad the image
    img_resized = tf.image.resize_with_crop_or_pad(img, target_size[0], target_size[1])
    # Remove the extra dimension
    img_resized = tf.squeeze(img_resized, axis=0)
    # Convert the image back to PIL format
    img_array = image.array_to_img(img_resized)
    # ----------------------------------------------------
    
    # img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    images.append(img_array)
    image_list.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model2.predict(images, batch_size=10)

# Convert predictions to class names based on the threshold
class_names_predicted = ['Parasitized' if classes < 0.5
                         else 'Uninfected' for classes in classes]

# print(classes)

In [ ]:
# Plot the results
# ===================================================================

# Plot the images along with their predicted classes
fig = plt.figure(figsize=(10, 10))
for i in range(min(20, len(image_list))):  # Limit the number of images to 10
    ax = fig.add_subplot(5, 4, i+1)
    ax.imshow(images[i])
    ax.set_title(class_names_predicted[i])
    ax.yaxis.set_visible(False)
    ax.xaxis.set_ticks([])
plt.tight_layout()
plt.show()